In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from ncps.torch import LTC

# ---- DATA PREP SAME AS BEFORE ----
data = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

feature_cols = ['year', 'month', 'day', 'hour', 'min', 'sec',
                'lat', 'lon', 'depth', 'class']

time_target_cols = ['year_as', 'day_as', 'hour_as', 'min_as', 'sec_as', 'lat_as', 'lon_as']

test[time_target_cols] = np.nan

inputs = data[feature_cols].values
targets = data[time_target_cols].values

test_inputs = test[feature_cols].values
test_targets = test[time_target_cols].values

input_scaler = StandardScaler()
target_scaler = StandardScaler()
inputs_scaled = input_scaler.fit_transform(inputs)
targets_scaled = target_scaler.fit_transform(targets)

test_inputs_scaled = input_scaler.fit_transform(test_inputs)
test_targets_scaled = test_targets

X_train, X_val, y_train, y_val = train_test_split(
    inputs_scaled, targets_scaled, test_size=0.4, random_state=42)

class TimeSeriesDataset(Dataset):
    def __init__(self, inputs, targets, latlon_targets=None):
        self.inputs = torch.tensor(inputs, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)
        self.latlon_targets = torch.tensor(latlon_targets, dtype=torch.float32) if latlon_targets is not None else None

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        if self.latlon_targets is not None:
            return self.inputs[idx], self.targets[idx], self.latlon_targets[idx]
        else:
            return self.inputs[idx], self.targets[idx]

train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset   = TimeSeriesDataset(X_val,   y_val)
test_dataset  = TimeSeriesDataset(test_inputs_scaled,  test_targets_scaled)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ----- MODEL DEFINITIONS -----
# 1. LTC
class LTCPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LTCPredictor, self).__init__()
        self.rnn = LTC(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        output = self.fc(rnn_out)
        return output

# 2. MLP
class MLPPredictor(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers=[128,64]):
        super().__init__()
        layers = []
        prev = input_size
        for h in hidden_layers:
            layers.extend([nn.Linear(prev, h), nn.ReLU()])
            prev = h
        layers.append(nn.Linear(prev, output_size))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

# 3. LSTM
class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# 4. PINN (Physics-informed NN). For example, penalize deviation from Omori law.
class PINNPredictor(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers=[128,64]):
        super().__init__()
        layers = []
        prev = input_size
        for h in hidden_layers:
            layers.extend([nn.Linear(prev, h), nn.ReLU()])
            prev = h
        layers.append(nn.Linear(prev, output_size))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

    def physics_loss(self, predictions, Omori_exp=1.0, weight=1.0):
        # Dummy: Penalize predictions that do not decay in time (Omori: N(t)~1/(t+c)^p)
        # Here we use aftershock time (in hours) as predictions[...,1]*60+predictions[...,2]
        hours = predictions[...,1].detach() + predictions[...,2].detach()/60.
        penalty = torch.mean(torch.relu(hours - 24))  # Example: penalty if predicted aftershock is >24h (should typically decay; this is a simplification)
        return weight * penalty

# 5. LTC + PINN
class LTC_PINN_Predictor(LTCPredictor):
    # Inherit from LTC, add physics loss
    def physics_loss(self, predictions, Omori_exp=1.0, weight=1.0):
        # Same as above for demonstration
        hours = predictions[...,1].detach() + predictions[...,2].detach()/60.
        penalty = torch.mean(torch.relu(hours - 24))
        return weight * penalty

# --- TRAINING AND EVALUATION WRAPPER ---
def train_and_evaluate(model_name, num_epochs=30):
    input_size = 10
    output_size = 6
    learning_rate = 0.001
    hidden_size = 64

    # Chose model
    if model_name == "LTC":
        model = LTCPredictor(input_size, hidden_size, output_size)
    elif model_name == "MLP":
        model = MLPPredictor(input_size, output_size)
    elif model_name == "LSTM":
        model = LSTMPredictor(input_size, hidden_size, output_size)
    elif model_name == "PINN":
        model = PINNPredictor(input_size, output_size)
    elif model_name == "LTC_PINN":
        model = LTC_PINN_Predictor(input_size, hidden_size, output_size)
    else:
        raise ValueError(f"Unknown model {model_name}")

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for batch in train_loader:
            inputs, targets = batch

            # MLP treats inputs as (batch, features), others as (batch, seq, features)
            if model_name == "MLP" or model_name == "PINN":
                outputs = model(inputs)
            else:
                outputs = model(inputs.unsqueeze(1)).squeeze(1)
            loss = criterion(outputs, targets)
            # Add physics-informed penalty if required
            if model_name == "PINN" or model_name == "LTC_PINN":
                loss += model.physics_loss(outputs, weight=0.5)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        # Validation loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                inputs, targets = batch
                if model_name == "MLP" or model_name == "PINN":
                    outputs = model(inputs)
                else:
                    outputs = model(inputs.unsqueeze(1)).squeeze(1)
                loss = criterion(outputs, targets)
                if model_name == "PINN" or model_name == "LTC_PINN":
                    loss += model.physics_loss(outputs, weight=0.5)
                val_loss += loss.item()
        # print(f"{model_name} Epoch [{epoch+1}], Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")

    # ----- EVALUATION -----
    model.eval()
    y_preds = []
    y_trues = []
    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = batch
            if model_name == "MLP" or model_name == "PINN":
                outputs = model(inputs)
            else:
                outputs = model(inputs.unsqueeze(1)).squeeze(1)
            y_preds.append(outputs.numpy())
            y_trues.append(targets.numpy())
    y_preds = np.concatenate(y_preds, axis=0)
    y_preds_orig = target_scaler.inverse_transform(y_preds)
    
    return {"Model": model_name, "y_preds": y_preds_orig}

# --- RUN ALL MODELS AND AGGREGATE ---
model_names = ["LTC", "PINN", "LTC_PINN", "MLP", "LSTM"]
results = []
for m in model_names:
    print(f"Training: {m}")
    results.append(train_and_evaluate(m, num_epochs=25)) # Reduce for speed if needed

results_df = pd.DataFrame(results)
print("\n=== Model Performance Overview ===")
print(results_df)

Training: LTC
Training: PINN
Training: LTC_PINN
Training: MLP
Training: LSTM

=== Model Performance Overview ===
      Model                                            y_preds
0       LTC  [[17.345795, 15.078236, 31.49224, 303.61502, 4...
1      PINN  [[15.874502, 14.318308, 30.072746, 380.83118, ...
2  LTC_PINN  [[16.45072, 14.353564, 31.582644, 298.10837, 4...
3       MLP  [[19.265553, 13.58434, 35.973076, 405.7473, 40...
4      LSTM  [[15.284285, 17.5536, 33.72706, 334.20267, 40....


In [46]:
results_df['y_preds']

0    [[17.345795, 15.078236, 31.49224, 303.61502, 4...
1    [[15.874502, 14.318308, 30.072746, 380.83118, ...
2    [[16.45072, 14.353564, 31.582644, 298.10837, 4...
3    [[19.265553, 13.58434, 35.973076, 405.7473, 40...
4    [[15.284285, 17.5536, 33.72706, 334.20267, 40....
Name: y_preds, dtype: object

In [48]:
answer = pd.DataFrame()
id = pd.read_csv("data/test.csv")
answer['id_eq'] = id['id_eq']
answer[time_target_cols] = results_df['y_preds'][0]
answer

,id_eq,day_as,hour_as,min_as,sec_as,lat_as,lon_as
0,996,17.345795,15.078236,31.492241,303.615021,40.690990,75.775551
1,1336,6.044427,7.720076,31.690807,271.540436,39.575855,73.512192
2,730,17.896954,13.469764,29.067022,297.230438,44.595146,81.704361
3,696,22.706982,15.972118,30.921684,290.349152,39.685047,72.893486
4,1460,21.196991,13.734555,27.097343,288.634857,43.606991,80.571281
...,...,...,...,...,...,...,...
295,1422,10.653868,7.172418,30.806149,291.272736,40.623516,72.539696
296,737,22.439529,9.695239,25.260485,342.637115,40.237141,73.839867
297,1737,20.409685,10.230364,28.854362,266.423706,38.387657,73.380913
298,1184,22.907253,14.192774,27.314438,312.193665,39.347271,73.714424


In [13]:
tdata = pd.read_csv("./test (2).csv")
tdata2 = pd.read_csv("./data/test.csv")

tdata2[['year_as', 'month_as', 'lat_as', 'lon_as', 'depth_as', 'class_as']] = tdata[['year_as', 'month_as', 'lat_as', 'lon_as', 'depth_as', 'class_as']] 

In [14]:
tdata = tdata2
X = tdata[['year', 'month', 'day', 'hour', 'min', 'sec', 'lat', 'lon', 'depth', 'class', 'year_as', 'month_as', 'lat_as', 'lon_as', 'depth_as', 'class_as']].values
def predict(input_data):
    scaled = input_scaler.transform(input_data)
    with torch.no_grad():
        output = model(torch.tensor(scaled, dtype=torch.float32).unsqueeze(1))
    return target_scaler.inverse_transform(output.squeeze(1).numpy())
predictions = predict(X)

predictions

NameError: name 'model' is not defined

In [ ]:
response = pd.DataFrame()
response['id_eq']=tdata['id_eq']
response[['day_as', 'hour_as', 'min_as', 'sec_as']] = predictions

response

,id_eq,day_as,hour_as,min_as,sec_as
0,996,19.199181,15.710058,30.006195,291.872101
1,1336,9.769760,11.140163,31.982504,247.276535
2,730,15.819298,13.293402,29.263823,238.972748
3,696,20.961168,15.900433,29.641821,240.996445
4,1460,19.140297,13.308818,27.987635,233.776688
...,...,...,...,...,...
295,1422,11.931689,7.715176,28.591518,268.100922
296,737,21.075644,13.579777,26.790846,304.771759
297,1737,21.920063,11.468291,28.955589,224.258850
298,1184,21.760019,15.446991,27.406750,262.928070


In [ ]:
response.to_csv("ltc.csv", index=False)

: 